In [1]:
import time
from osgeo import gdal, osr
import numpy
import subprocess
from subprocess import PIPE, Popen
import os
from numpy.core.umath import add, subtract
import gc
import sys
sys.path.append('/usr/bin/')
import gdal_merge as gm
from multiprocessing import Pool 
class MDimage(object):
    def __init__(self, filepath):
        self.filepath = filepath
        self.ds = gdal.Open(filepath, gdal.GA_ReadOnly)
        self.iminfo = dict()
        self.iminfo['bandnum'] = self.ds.RasterCount
        self.iminfo['cols'] = self.ds.RasterXSize
        self.iminfo['rows'] = self.ds.RasterYSize
        self.iminfo['originX'] = self.ds.GetGeoTransform()[0]
        self.iminfo['originY'] = self.ds.GetGeoTransform()[3]
        self.iminfo['pixelWidth'] = self.ds.GetGeoTransform()[1]
        self.iminfo['pixelHeight'] = self.ds.GetGeoTransform()[5]

class Index(MDimage):
    def __init__(self, filepath):
        self.filepath = filepath
        super(Index, self).__init__(filepath)

    def Band2Array(self,fill):
        cols = self.iminfo['cols']
        rows = self.iminfo['rows']
        band = self.ds.GetRasterBand(1)
        array = band.ReadAsArray(0, 0, cols, rows)
        #array = array.reshape(cols * rows)
        return array
        
    def WriteArrayAsImage(self, out_fname, outArray):
        cols = self.iminfo['cols']
        rows = self.iminfo['rows']
        driver = self.ds.GetDriver()
        outArray= outArray.reshape([rows,cols])
        outDS = driver.Create(out_fname, cols, rows, 1, gdal.GDT_Int16)
        outDS.SetGeoTransform(self.ds.GetGeoTransform())
        outDS.SetProjection(self.ds.GetProjection())
        outBand = outDS.GetRasterBand(1)
        outBand.WriteArray(outArray)
        outDS = None
        del outDS, outBand
    
    def FileCheckDel(self, filepath):
        if os.path.exists(filepath):
            os.remove(filepath)
            
    def ClipWithGdalwarp(self, infile, outfile, shape):
        self.FileCheckDel(outfile)
        layer = os.path.split(os.path.splitext(shape)[0])[1]
        command = ['gdalwarp', '-of', 'GTiff', '-cutline', shape, '-cl',\
                   layer, '-crop_to_cutline', infile, outfile]
        command = [comm.encode('utf-8') for comm in command]
        p = subprocess.Popen(command, \
                            stdin=PIPE, \
                            stdout=PIPE, \
                            stderr=PIPE)
        p.wait()
        print "stdout: %s" % (p.stdout.readlines(), )
        #print "Erros: %s" % (p.stderr.readlines(), )
        
    
    def MergeWithGdalwarp(self, infile, outfile):
        self.FileCheckDel(outfile)
        NO_DATA_VALUE = '-999'
        command = ['gdal_merge.py', '-o', outfile,'-n', NO_DATA_VALUE,
         infile[0], infile[1], infile[2], infile[3]]
        command = [comm.encode('utf-8') for comm in command]
        p = subprocess.Popen(command, \
                            stdin=PIPE, \
                            stdout=PIPE, \
                            stderr=PIPE)
        p.wait()
        #print "stdout: %s" % (p.stdout.readlines(), )
        #print "Erros: %s" % (p.stderr.readlines(), )
        
    def Merge(self, infile, outfile):
        sys.argv = [ infile[0], infile[1], infile[2], infile[3], '-o',outfile, '-a_nodata', '-999']
        gm.main()

In [3]:
limage = Index(files[0])

In [4]:
fill=-999
year=2010
shape="/home/faizan/Pakistan/pakistan shape file/PAK_adm0.shp"

files=[]
for i in range(1, 362):
    files=[]
    for V in range(5,7):
        for H in range(23,25):
            files.append("/home/faizan/Pakistan/EVI/2010/MODIS.EVI.%d%03d"  % (year, i)+ ".h%dv%02d" % (H,V) + ".tif" )
    merge="/home/faizan/Pakistan/EVI/2010/mosaic.%d%03d"  % (year, i) + ".tif"
    limage.MergeWithGdalwarp(files, merge)
    clip="/home/faizan/Pakistan/EVI/2010/clip.%d%03d"  % (year, i) + ".tif"
    limage.ClipWithGdalwarp(merge, clip, shape)

stdout: ['Creating output file that is 3389P x 2679L.\n', 'Processing input file /home/faizan/Pakistan/EVI/2010/mosaic.2010362.tif.\n', '0...10...20...30...40...50...60...70...80...90...100 - done.\n']


In [167]:
limage = Index(files[0])

In [168]:
#limage.Merge(files,out)

In [169]:
limage.MergeWithGdalwarp(files, out)

stdout: ['0...10...20...30...40...50...60...70...80...90...100 - done.\n']
Erros: []


In [170]:
infile="/home/faizan/Desktop/mosaic1.tif"
outfile="/home/faizan/Desktop/pak2003001.tif"
shape="/home/faizan/Pakistan/pakistan shape file/PAK_adm0.shp"

limage.ClipWithGdalwarp(infile, outfile, shape)

In [56]:
fi

'/home/faizan/Desktop/MODIS.EVI.2003001.h23v05.tif /home/faizan/Desktop/MODIS.EVI.2003001.h24v05.tif /home/faizan/Desktop/MODIS.EVI.2003001.h23v06.tif /home/faizan/Desktop/MODIS.EVI.2003001.h24v06.tif'

NameError: name 'gdalwrap' is not defined

In [19]:
fill=-999
year=2003
files=[]
for i in range(1, 2):
    for V in range(5,7):
        for H in range(23,25):
            files.append("/home/faizan/Desktop/MODIS.EVI.%d%03d"  % (year, i)+ ".h%dv%02d" % (H,V) + ".tif" )

#for l in files:
limage = Index(files[0])
cols = limage.iminfo['cols']
rows = limage.iminfo['rows']
bandnum= limage.iminfo['bandnum']
h23v05 = (limage.Band2Array(fill))

limage = Index(files[1])
cols = limage.iminfo['cols']
rows = limage.iminfo['rows']
bandnum= limage.iminfo['bandnum']
h24v05 = (limage.Band2Array(fill))

Data1 = numpy.hstack([h23v05, h24v05])

limage = Index(files[2])
cols = limage.iminfo['cols']
rows = limage.iminfo['rows']
bandnum= limage.iminfo['bandnum']
h23v06 = (limage.Band2Array(fill))

limage = Index(files[3])
cols = limage.iminfo['cols']
rows = limage.iminfo['rows']
bandnum= limage.iminfo['bandnum']
h24v06 = (limage.Band2Array(fill))

Data2 = numpy.hstack([h23v06, h24v06])

Data = numpy.vstack([Data1, Data2])
        




ValueError: all the input array dimensions except for the concatenation axis must match exactly

In [37]:
h23v06.shape



(2000, 3215)

In [31]:
limage = Index("/home/faizan/Desktop/merge_pak.tif")
cols = limage.iminfo['cols']
rows = limage.iminfo['rows']
bandnum= limage.iminfo['bandnum']
merge = (limage.Band2Array(fill))

In [33]:
merge.shape

(4000, 7635)

In [9]:
files

['/home/ken/Data/EVI/MODIS.EVI.2000091.tif.Illinois.tif',
 '/home/ken/Data/EVI/MODIS.EVI.2000092.tif.Illinois.tif',
 '/home/ken/Data/EVI/MODIS.EVI.2000093.tif.Illinois.tif',
 '/home/ken/Data/EVI/MODIS.EVI.2000094.tif.Illinois.tif',
 '/home/ken/Data/EVI/MODIS.EVI.2000095.tif.Illinois.tif',
 '/home/ken/Data/EVI/MODIS.EVI.2000096.tif.Illinois.tif',
 '/home/ken/Data/EVI/MODIS.EVI.2000097.tif.Illinois.tif',
 '/home/ken/Data/EVI/MODIS.EVI.2000098.tif.Illinois.tif',
 '/home/ken/Data/EVI/MODIS.EVI.2000099.tif.Illinois.tif',
 '/home/ken/Data/EVI/MODIS.EVI.2000100.tif.Illinois.tif',
 '/home/ken/Data/EVI/MODIS.EVI.2000101.tif.Illinois.tif',
 '/home/ken/Data/EVI/MODIS.EVI.2000102.tif.Illinois.tif',
 '/home/ken/Data/EVI/MODIS.EVI.2000103.tif.Illinois.tif',
 '/home/ken/Data/EVI/MODIS.EVI.2000104.tif.Illinois.tif',
 '/home/ken/Data/EVI/MODIS.EVI.2000105.tif.Illinois.tif',
 '/home/ken/Data/EVI/MODIS.EVI.2000106.tif.Illinois.tif',
 '/home/ken/Data/EVI/MODIS.EVI.2000107.tif.Illinois.tif',
 '/home/ken/Da